# **MovieLens (small) Dataset**
**In This notebook we explore and preprocess Data** <br>
***Our Team:***<br>
- Sarah Sameh Elzahaby
- Ola Adel Hussien
- Nawal Mahmoud Shehata
- Lina Mohamed Noureldin
- Marina Magdy Ramzy<br>

ITI_ALEX_Bransh

## Install Required Libraries

### `1.` Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import re
import glob
import os
import datetime
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer


from sklearn.preprocessing import OneHotEncoder

## `2.` Loading Datasets

In [ ]:
# Define the directory containing the CSV files
directory = '/content/ml-latest-small'

# List all files in the directory
files = os.listdir(directory)

# Dictionary to store DataFrames
dataframes = {}
for file in files:
    if file.endswith('.csv'):
        filepath = os.path.join(directory, file)
        df_name = file.split('.')[0]
        dataframes[df_name] = pd.read_csv(filepath)

In [ ]:
# Assuming the files are named 'movies.csv', 'ratings.csv', 'tags.csv', and 'links.csv'
movies_df = dataframes['movies']
ratings_df = dataframes['ratings']
tags_df = dataframes['tags']
links_df = dataframes['links']

## `i.` Movies dataframe

In [ ]:
# Print the first 3 rows of the movies_df DataFrame
movies_df.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


### Check Nulls

In [ ]:
null_count = movies_df.isnull().sum()
null_percentage = (movies_df.isnull().sum() / len(movies_df)) * 100
null_info = pd.DataFrame({'Null Count': null_count, 'Null Percentage': null_percentage})
null_info

,Null Count,Null Percentage
movieId,0,0.0
title,0,0.0
genres,0,0.0


In [ ]:
print('Unique movies:', movies_df['movieId'].nunique())

Unique movies: 9742


### Preprocess on Movie Table
### **split year from movie title**

In [ ]:
def split_title_and_year(title: str) -> tuple[str, str]:
    """
    Split a movie title into its name and year.

    Parameters:
    title (str): The title of the movie.

    Returns:
    tuple[str, str]: A tuple containing the name of the movie and its release year.
                     If the year is not found, the second element of the tuple is None.
    """
    # Use regular expression to find the year in parentheses
    match = re.search(r'\((\d{4})\)', title)

    # If year is found, extract it and strip the name
    if match:
        year = match.group(1)
        name = title[:match.start()].strip()
        return name, year

    # If year is not found, return the title and None for year
    return title, None

In [ ]:
#Apply Function
movies_df[['title', 'year']] = movies_df['title'].apply(lambda x: pd.Series(split_title_and_year(x)))
movies_df

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,Action|Animation|Comedy|Fantasy,2017
9738,193583,No Game No Life: Zero,Animation|Comedy|Fantasy,2017
9739,193585,Flint,Drama,2017
9740,193587,Bungo Stray Dogs: Dead Apple,Action|Animation,2018


### **split genres**

In [ ]:
# Split the genres column by '|'
movies_df['genres'] = movies_df['genres'].str.split('|')
movies_df.head(3)

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995


In [ ]:
# Find unique genres
unique_genres = set(genre for sublist in movies_df['genres'] for genre in sublist)
unique_genres

{'(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

### Handel Genres

In [ ]:
for genre in unique_genres:
    movies_df[genre] = 0
# Loop through each movie and set the genre columns to 1 if the movie has that genre
for idx, row in movies_df.iterrows():
    for genre in row['genres']:
        movies_df.at[idx, genre] = 1

# Drop the original 'genres' column as it is no longer needed
movie_df = movies_df.drop(columns=['genres'])
movie_df

,movieId,title,year,Children,Western,(no genres listed),Horror,Fantasy,Romance,Thriller,...,Comedy,Film-Noir,Animation,IMAX,War,Crime,Mystery,Action,Documentary,Adventure
0,1,Toy Story,1995,1,0,0,0,1,0,0,...,1,0,1,0,0,0,0,0,0,1
1,2,Jumanji,1995,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,3,Grumpier Old Men,1995,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale,1995,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II,1995,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,2017,0,0,0,0,1,0,0,...,1,0,1,0,0,0,0,1,0,0
9738,193583,No Game No Life: Zero,2017,0,0,0,0,1,0,0,...,1,0,1,0,0,0,0,0,0,0
9739,193585,Flint,2017,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9740,193587,Bungo Stray Dogs: Dead Apple,2018,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0


## `ii.` ratings dataframe

In [ ]:
# Print the first 3 rows of the rating_df DataFrame
ratings_df.head(3)

,userId,movieId,rating,timestamp
0,1,1,4.0,2000-07-30 18:45:03
1,1,3,4.0,2000-07-30 18:20:47
2,1,6,4.0,2000-07-30 18:37:04


### Check Nulls

In [ ]:
null_count = ratings_df.isnull().sum()
null_percentage = (ratings_df.isnull().sum() / len(ratings_df)) * 100
null_info = pd.DataFrame({'Null Count': null_count, 'Null Percentage': null_percentage})
null_info

,Null Count,Null Percentage
userId,0,0.0
movieId,0,0.0
rating,0,0.0
timestamp,0,0.0


In [ ]:
print('Unique movies:', ratings_df['userId'].nunique())
print('Unique movies:', ratings_df['movieId'].nunique())

Unique movies: 610
Unique movies: 9724


### Handel Timestamp

**Convert ratings timestamp (unix) to number of months passed until 24-9-2018**

In [ ]:
# Convert Unix timestamp to datetime
ratings_df['timestamp'] = pd.to_datetime(ratings_df['timestamp'], unit='s')
ratings_df.head(3)

,userId,movieId,rating,timestamp
0,1,1,4.0,2000-07-30 18:45:03
1,1,3,4.0,2000-07-30 18:20:47
2,1,6,4.0,2000-07-30 18:37:04


### Convert it Numbers of Months

In [ ]:
# Define the target date
target_date = datetime.datetime(2018, 9, 24)
# Calculate the number of months passed until the target date
ratings_df['months_passed_on_rating'] = (target_date.year - ratings_df['timestamp'].dt.year) * 12 + (target_date.month - ratings_df['timestamp'].dt.month)
ratings_df.drop(columns='timestamp', inplace=True)
ratings_df.head(3)

,userId,movieId,rating,months_passed_on_rating
0,1,1,4.0,218
1,1,3,4.0,218
2,1,6,4.0,218


## `iii.` Tags dataframe

In [ ]:
# Print the first 3 rows of the tags_df DataFrame
tags_df.head(3)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992


### Check Nulls

In [ ]:
null_count = tags_df.isnull().sum()
null_percentage = (tags_df.isnull().sum() / len(tags_df)) * 100
null_info = pd.DataFrame({'Null Count': null_count, 'Null Percentage': null_percentage})
null_info

,Null Count,Null Percentage
userId,0,0.0
movieId,0,0.0
tag,0,0.0
timestamp,0,0.0


### Handel Timestamp

**Convert tags timestamp to number of months until 24-9-2018**

In [ ]:
# Convert Unix timestamp to datetime
tags_df['timestamp'] = pd.to_datetime(tags_df['timestamp'], unit='s')
tags_df. head(3)

,userId,movieId,tag,timestamp
0,2,60756,funny,2015-10-24 19:29:54
1,2,60756,Highly quotable,2015-10-24 19:29:56
2,2,60756,will ferrell,2015-10-24 19:29:52


In [ ]:
# Calculate the number of months passed until the target date
tags_df['months_passed_on_tags'] = (target_date.year - tags_df['timestamp'].dt.year) * 12 + (target_date.month - tags_df['timestamp'].dt.month)
tags_df.drop(columns='timestamp', inplace=True)
tags_df.head(3)

,userId,movieId,tag,months_passed_on_tags
0,2,60756,funny,35
1,2,60756,Highly quotable,35
2,2,60756,will ferrell,35


## `iv.` links dataframe

In [ ]:
# Print the first 3 rows of the links_df DataFrame
links_df.head(3)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0


##`v.` Merge important datasets (links will not be needed)

In [ ]:
# Merge ratings_df with movies_df on movieId (left join to preserve all ratings)

merged_df = pd.merge(ratings_df, movies_df, on='movieId')

merged_df.shape

(100836, 27)

In [ ]:
# Merge the result with tags_df on userId and movieId (left join to preserve all ratings)
df = pd.merge(merged_df, tags_df[['userId', 'movieId', 'tag', 'months_passed_on_tags']], on=['userId', 'movieId'], how='left')
df.shape

(102677, 29)

In [ ]:
df.drop(columns='genres', inplace=True)

##`VI` Data preprocessing after Merging

### Check Nulls after marge

In [ ]:
null_count = df.isnull().sum()
null_percentage = (df.isnull().sum() / len(df)) * 100
null_info = pd.DataFrame({'Null Count': null_count, 'Null Percentage': null_percentage})
null_info

,Null Count,Null Percentage
userId,0,0.000000
movieId,0,0.000000
rating,0,0.000000
months_passed_on_rating,0,0.000000
title,0,0.000000
year,20,0.019479
Children,0,0.000000
Western,0,0.000000
(no genres listed),0,0.000000
Horror,0,0.000000


## `VII.` vectorize tags using TF-IDF

In [ ]:
tfidf = TfidfVectorizer()
df['tag'] = df['tag'].fillna('')

In [ ]:
tags_tfidf = tfidf.fit_transform(df['tag']).toarray()
tags_tfidf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
# Convert the tag matrix to a DataFrame
tags_tfidf = pd.DataFrame(tags_tfidf, columns=tfidf.get_feature_names_out())
tags_tfidf.head(3)

,06,1900s,1920s,1950s,1960s,1970s,1980s,1990s,2001,250,...,york,you,younger,your,zellweger,zither,zoe,zombie,zombies,zooey
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Drop the original 'tag' column
df.drop(columns=['tag'], inplace=True)

In [ ]:
# Concatenate the tag DataFrame with the original DataFrame
df_sparse = pd.concat([df, tags_tfidf], axis=1)
df_sparse

,userId,movieId,rating,months_passed_on_rating,title,year,Children,Western,(no genres listed),Horror,...,york,you,younger,your,zellweger,zither,zoe,zombie,zombies,zooey
0,1,1,4.0,218,Toy Story,1995,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,1,4.0,262,Toy Story,1995,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7,1,4.5,164,Toy Story,1995,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15,1,2.5,10,Toy Story,1995,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,17,1,4.5,88,Toy Story,1995,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102672,610,160341,2.5,22,Bloodmoon,1997,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102673,610,160527,4.5,22,Sympathy for the Underdog,1971,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102674,610,160836,3.0,16,Hazard,2005,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102675,610,163937,3.5,16,Blair Witch,2016,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_sparse = df_sparse.rename(columns={'userId': 'user', 'movieId': 'item', 'time':'timing', 'rating_months_passed':'time'})

## final dataframe

In [ ]:
df_sparse.head(3)

,user,item,rating,months_passed_on_rating,title,year,Children,Western,(no genres listed),Horror,...,york,you,younger,your,zellweger,zither,zoe,zombie,zombies,zooey
0,1,1,4.0,218,Toy Story,1995,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,1,4.0,262,Toy Story,1995,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7,1,4.5,164,Toy Story,1995,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_sparse.to_csv('/content/movies_rs.csv', index=False)